## Vectorización IDEAM

In [1]:
from osgeo import gdal, ogr
import sys
import os
import numpy as np
import fiona
import rasterio
import rasterio.mask
os.environ['USE_PYGEOS'] = '0'
from rasterio.features import shapes
from shapely.geometry import shape
import shapely 
import shapely.speedups
import geopandas as gpd
import pandas as pd
import time
import re

In [2]:
ideam_dir='/Users/santiagoherreragarcia/Library/CloudStorage/OneDrive-Bibliotecascompartidas:Universidaddelosandes/Jeimy Carolina Hormizda Fonseca - Cambio de cobertura 16_06_2023/IDEAM IMG'
vector_dir='/Users/santiagoherreragarcia/Library/CloudStorage/OneDrive-Bibliotecascompartidas:Universidaddelosandes/Jeimy Carolina Hormizda Fonseca - Cambio de cobertura 16_06_2023/IDEAM Vectorizada (Deforestacion)/Points Vector (pixeles 1 y 2)'
pacifico_tifs='/Users/santiagoherreragarcia/Library/CloudStorage/OneDrive-Bibliotecascompartidas:Universidaddelosandes/Jeimy Carolina Hormizda Fonseca - Cambio de cobertura 16_06_2023/Pacifico Tifs'

In [3]:
os.chdir(ideam_dir)

#Guardamos todos los archivos de Hansen que abarcan Colombia en una lista.
defo_files=[]
for i in os.listdir():
    if '.img' not in i:
        continue
    else:
        data=rasterio.open(i, 'r')
        defo_files.append(data)

In [4]:
#Los otros datos importantes están en el shapefile del Pacífico
pacifico_gdf=gpd.read_file("/Users/santiagoherreragarcia/Library/CloudStorage/OneDrive-Bibliotecascompartidas:Universidaddelosandes/Carolina Castro Osorio - REDD+ bases/Inputs/pacifico_disuelto")

In [5]:
#Creamos nuevos tiffs con la info del Pacífico únicamente
for f in defo_files:
    project=re.search('(.*?)(?=.img)', f.name).group(0).replace(' ', '_')
    file_name='/pacifico_ideam_{}.tif'.format(project)

    #Usar mask del Pacifico 
    pacifico_reproj=pacifico_gdf.to_crs(f.crs)
    masked_raster, masked_raster_transform = rasterio.mask.mask(f, pacifico_reproj[['geometry']].values.flatten(), crop=True)

    #Escribir raster
    f_meta=f.meta
    f_meta.update({"driver": "GTiff",
                 "height": masked_raster.shape[1],
                 "width": masked_raster.shape[2],
                 "transform": masked_raster_transform})

    with rasterio.open(pacifico_tifs+"/"+file_name, "w", **f_meta) as dest:
        dest.write(masked_raster)

In [3]:
#Ahora leemos los tiffs con mask y los vectorizamos
defo_masked_files=[]
os.chdir(pacifico_tifs)
for i in os.listdir():
    if '.tif' not in i:
        continue
    else:
        data=rasterio.open(i, 'r')
        defo_masked_files.append(data)

In [4]:
start = time.time()

for m in defo_masked_files:
    project=re.search('(.*?)(?=.tif)', m.name).group(0).replace(' ', '_')
    file_name='/Points_{}.parquet'.format(project)

    raster_pacifico=m.read(1)

    # Obtener las coordenadas de los puntos con valor mayor a cero
    points = np.argwhere((raster_pacifico==2))
    
    # Dejar la columna de dato
    values = raster_pacifico[points[:, 0], points[:, 1]]
    

    # Convertir las coordenadas de píxeles a coordenadas espaciales
    raster_transform=m.transform
    
    lon, lat = rasterio.transform.xy(transform=raster_transform, rows=points[:, 0], cols=points[:, 1])
    print(len(values))

    # Crear un GeoDataFrame de los puntos
    gdf = gpd.GeoDataFrame({"geometry":gpd.points_from_xy(lon, lat), "pixel_type":values})
    print("Sin error")
    gdf.crs = m.crs
    gdf=gdf.to_crs(epsg=3116)

    #Definir columna de año
    pattern = r'(\d{4})[_-](\d{4})' 
    matches = re.findall(pattern, m.name)
    if len(matches) >= 1:
        año = matches[0][1]

    gdf["año"]=año

    gdf.to_parquet(vector_dir+file_name)

    print("Listo "+str(año)+": "+str(len(gdf))+"obs")

end = time.time()

print(end-start)

89522447
Sin error


: 

: 

os.chdir(vector_dir)
points_2005=gpd.read_parquet('Points_cambio_2000_2005_v5_140319.parquet')
points_2019=gpd.read_parquet('Points_cambio_2018_2019_v8_200707.parquet')
points_2014=gpd.read_parquet('Points_cambio_2013_2014_V6_151015.parquet')

points_2005.to_file('/Users/santiagoherreragarcia/Desktop/points2005.gpkg')
points_2019.to_file('/Users/santiagoherreragarcia/Desktop/points2019.gpkg')
points_2014.to_file('/Users/santiagoherreragarcia/Desktop/points2014.gpkg')